In [1]:
import pynbody
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
galaxy_masses=pd.DataFrame()
BH_halos=pd.DataFrame()
snap_id=0

In [3]:
iords=pd.read_csv('csvdata/iords.csv',index_col='snap')
masses=pd.read_csv('csvdata/ALL_galaxy_masses.csv',index_col='snap')

In [4]:
def loader(path):
    '''returns snapshot and halo'''
    snap=pynbody.load(path)
    snap.physical_units()
    h=snap.halos()
    return snap,h

def BH_finder(snap):
    '''returns subsnap of black holes'''
    return snap.star[pynbody.filt.BandPass('tform','-15 Gyr','0 Gyr')]

def analysis(halo, view):
    '''center snap on halo's center of mass (angmom gives errors if there aren't enough pixels)'''
    if view==1:
        pynbody.analysis.angmom.faceon(halo)
    elif view==2:
        pynbody.analysis.angmom.sideon(halo)
    else:
        pynbody.analysis.halo.center(halo)
        
def plot_BH_pos(h, BH, grpid, attribute, snapshot, cmap='Greys', w=100, save=False, view=0):
    '''plots position of black holes. Send halo array(h), black hole array(BH), grpid, attribute, and snapshot you want to plot 
    Optional: width/preferred scope(w) (default=100), colormap(default='Greys'), save(default=False)'''
    
    pynbody.plot.image(h[grpid].s,qty=attribute,cmap=cmap, width=w, log=False)
    plt.plot(np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[0],np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[1],'r+')
    plt.xlim(-w/2,w/2)
    plt.ylim(-w/2,w/2)
    plt.title('%s_h%s_%s_w%s'%(snapshot, grpid, attribute, w))
    
    if save==True:
        plt.savefig('plots/%s/h%s/h148_h%s_%s_w%s.png'%(snapshot, grpid, grpid, attribute, w))
    plt.show()
    
def add_mass_columns(hgrpid, grpid, snap_id):
    '''returns row with gas, stellar, and total masses for one halo tracked through time'''
    df=pd.DataFrame([[hgrpid.g['mass'].sum(), hgrpid.s['mass'].sum(), hgrpid['mass'].sum()]], index=[snap_id], columns=['H[%d] Gas'%(grpid),'H[%d] Star'%(grpid),'H[%d] Total'%(grpid)])
    df.units='Msol'
    return df

def mass_df(h, BHgrp, snap_id):
    df=pd.DataFrame()
    for i in BHgrp:
        df=pd.concat([df, add_mass_columns(h[i], i, snap_id)], axis=1, join='outer')
    
    df.units='Msol'
    return df

def galaxy_df(snap_id_array,h,BHgrp):
    galaxy_masses=pd.DataFrame()
    for i in snap_id_array:
        vars()['mass_%s'%(i)]=mass_df(h,BHgrp,i)
        galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(i)])

    return galaxy_masses

def plot_SFH(h, grpid,  snapshot, ls='dashed', c='k', save=True, mf=False):
    '''plots SFH. Send halo array(h[grpid]), linestyle(ls=dashed), color(c=k) grpid, and snapshot you want to plot 
    Optional:save(default=True)'''
    plt.clf()
    pynbody.plot.sfh(h[grpid].s,linestyle=ls,color=c, massform=mf);
    plt.title('%s_h[%s]_SFH'%(snapshot,grpid), y=1.14);
    plt.savefig('plots/SFH/%s_h[%s]_SFH.png'%(snapshot, grpid));
    

In [5]:
#array of last four digits for each snap
snap_id_array=['0139','0225','0275','0640','0974','1024','1269','1280','1408','1740','2048','2088','2432','2688','2816','2944','3072','3195','3200','3328','3456','3584','3606','3712','3840','3968','4096']

In [6]:
#snap id index (negative values start from end)
snap_id=snap_id-1
snap_id_array[snap_id]

'4096'

In [7]:
#set path
path='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id],snap_id_array[snap_id])

In [8]:
#returns snapshot and halos in physical units (takes a couple of minutes)
snap,h=loader(path)

In [ ]:
#pynbody.plot.sfh(h[0].s,linestyle='dashed',color='k', massform=False);

In [ ]:
#pynbody.analysis.halo.center(h[0])
#pynbody.analysis.angmom.sideon(h[0], cen=(0,0,0))

In [ ]:
#print 'Total: %s'%(len(h[0])), 'Gas: %s'%(len(h[0].g)), 'Stars: %s'%(len(h[0].s)), 'DM: %s'%(len(h[0].d))

In [ ]:
#pynbody.plot.image(h[0].g, width=100, cmap='Blues');

In [9]:
iords_list=iords.columns.tolist()

In [10]:
for i in iords_list:
    ind=0
    vars()['iord_%s'%(i)]=pd.DataFrame(columns=['gas', 'star', 'total'])
    iords_temp=iords[i].dropna()
    vars()['iord_%s'%(i)]['halo']=iords[i]
    for j in iords_temp.index:
        if pd.notnull(iords.at[j,i]):
            temp=[masses.at[j,'H[%s] Gas'%(int(iords.at[j,i]))],masses.at[j,'H[%s] Star'%(int(iords.at[j,i]))],masses.at[j,'H[%s] Total'%(int(iords.at[j,i]))]]
        if (pd.notnull(temp)).any():
            vars()['iord_%s'%(i)].loc[int(iords_temp.index[ind]),['gas','star','total']]=temp
        else: continue
        ind=ind+1
        vars()['iord_%s'%(i)]=vars()['iord_%s'%(i)].fillna(0)
        vars()['iord_%s'%(i)].to_csv('iord_%s.csv'%(i))

In [71]:
labels=[snap_id_array[0], snap_id_array[6],snap_id_array[13],snap_id_array[20],snap_id_array[26]]
n=0

In [88]:
x=iords_list[n]
plt.hist(range(0,27), weights=vars()['iord_%s'%(x)]['total'], bins=len(snap_id_array),width=.85, alpha=.3, histtype='bar', label='Total', color='#666666',rwidth=1, align='mid')
plt.hist([range(0,27),range(0,27)], weights=[vars()['iord_%s'%(x)]['gas'],vars()['iord_%s'%(x)]['star']],bins=len(snap_id_array),width=.85, alpha=1, histtype='bar', stacked=True, label=['Gas','Stars'],fill=True, color=['#44AA99','#88CCEE'],rwidth=1,align='mid')
#plt.hist(range(1,28), weights=vars()['iord_%s'%(x)]['star'], bins=len(snap_id_array),width=.9, alpha=1, histtype='barstacked', label='Stars',fill=True, color='gold',rwidth=0.5)
plt.title('Masses of Halo Containing iord %s'%(x));
plt.xlabel('Snap_id', fontsize='large');
plt.ylabel('Mass [M$_\odot$]', fontsize='large');
plt.xlim(-.5,26.5)
#plt.ylim(0,vars()['iord_%s'%(x)]['gas'].max()+10000000000)
plt.xticks([0.5,6.1,13,19.7,25.4],labels);
plt.legend()
plt.yscale('log')
#plt.savefig('plots/galaxy_masses/logscale_iord_%s_masses.png'%(x))
n+=1

IndexError: list index out of range

In [ ]:
path_higher_z='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id-1],snap_id_array[snap_id-1])

In [ ]:
snap

In [69]:
iord_1018


,gas,star,total,halo
snap,,,,
139,5.602900e+07,1.421747e+05,3.860902e+08,26
225,1.097535e+08,6.998123e+05,8.322149e+08,45
275,1.520925e+08,1.106535e+06,1.067258e+09,57
640,1.393357e+10,1.357321e+09,1.322043e+11,1
974,4.308672e+10,9.691792e+09,3.786796e+11,1
1024,4.677795e+10,1.181563e+10,4.162733e+11,1
1269,7.729830e+10,3.095780e+10,7.502674e+11,1
1280,7.945881e+10,3.191686e+10,7.691464e+11,1
1408,9.830424e+10,4.489980e+10,9.709940e+11,1


In [ ]:
iord_101867516

In [ ]:
plt.hist(range(1,27), weights=iord_101867516[1:]['total'], bins=len(snap_id_array),width=.85, alpha=.3, histtype='bar', label='Total', color='#666666',rwidth=1, align='mid')
plt.hist([range(1,27),range(1,27)], weights=[iord_101867516[1:]['gas'],iord_101867516[1:]['star']],bins=len(snap_id_array),width=.85, alpha=1, histtype='bar', stacked=True, label=['Gas','Stars'],fill=True, color=['#44AA99','#88CCEE'],rwidth=1,align='mid')
#plt.hist(range(1,28), weights=vars()['iord_%s'%(x)]['star'], bins=len(snap_id_array),width=.9, alpha=1, histtype='barstacked', label='Stars',fill=True, color='gold',rwidth=0.5)
plt.title('Masses of Halo Containing iord 101867516');
plt.xlabel('Snap_id', fontsize='large');
plt.ylabel('Mass [M$_\odot$]', fontsize='large');
plt.xlim(-.5,27.5)
#plt.ylim(0,vars()['iord_%s'%(x)]['gas'].max()+10000000000)
plt.xticks([0.5,6.5,13.5,20.5,26.5],labels);
plt.legend()
#plt.yscale('log')
plt.savefig('plots/galaxy_masses/iord_101867516_masses.png')

In [ ]:
iord_101867516.dropna()['total']